In [5]:
import os
import requests
import operator

from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.callbacks import get_openai_callback


os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = "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"
os.environ["LANGSMITH_PROJECT"] = "RRL project"

os.environ["AZURE_OPENAI_API_KEY"] = "EhMIoJnOsNomEJ8TRfOEoIc1jC49AwdEwmZ8UDi4lh6dsUZ4WEArJQQJ99BAACYeBjFXJ3w3AAAAACOGl3Rt"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://week31004687013.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview"


### Hotel Info

In [6]:
HOTEL_INFO = """Hilton Orrington / Evanston
3.5 starsBased on 380 reviews

1710 Orrington AveEvanstonIllinoisUnited States60201 See Map

ReservationDesk.com:866-215-2318
Hotel Description
Take advantage of recreation opportunities such as a 24-hour fitness center, or other amenities including complimentary wireless internet access and concierge services. Additional amenities at this hotel include wedding services, a fireplace in the lobby, and discounted use of a nearby fitness facility.

Enjoy a satisfying meal at Evanston Corner Bistro serving guests of Hilton Orrington / Evanston. Wrap up your day with a drink at the bar/lounge. Cooked-to-order breakfasts are served on weekdays from 7:30 AM to 10:00 AM and on weekends from 8:00 AM to 11:00 AM for a fee.

The property will be renovating from July 1 2024 to September 6 2024 (completion date subject to change). The following areas are affected: Lobby During renovations, the hotel will make every effort to minimize noise and disturbance.

Featured amenities include a business center, express check-in, and express check-out. Planning an event in Evanston? This hotel has 15756 square feet (1464 square meters) of space consisting of a conference center and meeting rooms.

Make yourself at home in one of the 269 guestrooms featuring refrigerators and flat-screen televisions. Your pillowtop bed comes with down comforters and premium bedding. Complimentary wireless internet access keeps you connected, and cable programming is available for your entertainment. Private bathrooms have designer toiletries and hair dryers.

You'll be centrally located in Evanston with a stay at Hilton Orrington / Evanston, within a 15-minute walk of Halim Time & Glass Museum and Clark Street Beach. This hotel is 0.7 mi (1.2 km) from Mary and Leigh Block Museum of Art and 1 mi (1.6 km) from Northwestern University.

Near Northwestern University

Bosnian, Arabic, Russian, English, Croatian, Romanian, Spanish, Serbian

Visa, Diners Club, Debit cards, Discover, Cash, American Express, JCB International, Mastercard

Hotel Policies

Know Before You Go

Only registered guests are allowed in the guestrooms. Parking height restrictions apply. This property welcomes guests of all sexual orientations and gender identities (LGBTQ+ friendly).


Age Restriction

Guests under the age of 21 can only check in with a parent or official guardian.



Check-In Instructions

Extra-person charges may apply and vary depending on property policy
Government-issued photo identification and a credit card, debit card, or cash deposit may be required at check-in for incidental charges
Special requests are subject to availability upon check-in and may incur additional charges; special requests cannot be guaranteed
This property accepts credit cards, debit cards, and cash
Long-term renters welcome
Safety features at this property include a fire extinguisher, a security system, and a first aid kit
Please note that cultural norms and guest policies may differ by country and by property; the policies listed are provided by the property

Check-In Special Instructions

Front desk staff will greet guests on arrival.


Fees

Optional

Fee for cooked-to-order breakfast: approximately USD 20 for adults and USD 20 for children
Covered valet parking fee: USD 33.00 per night (in/out privileges)
Pet fee: USD 75 per accommodation, per stay
Service animals are exempt from fees
In-room microwave fee: USD 25 per night
Rollaway bed fee: USD 25.0 per night
The above list may not be comprehensive. Fees and deposits may not include tax and are subject to change.

- Read less
Amenities
Internet Access
Spa
Fitness Center
Restaurant
Pets Allowed (conditions apply)
Room Service
Accessible
Meeting Facilities
Bar
Smoke free
Smoking Areas
24 hour
Concierge
Conference Center



Based on 380 User Reviews
Search reviews

Newest to Oldest

Traveler Type
4 stars review
Young Couple
Dec 2024
Ben
lobby re-do is great
rooms need a facelift - old and tired especially for parents weekend price!
5 stars review
Family with Younger Children
Dec 2024
Crystal
it was an easy walk to the university and downtown evanston. the room was huge, well equipped, and very comfortable.
nothing. it was great!
1 star review
Business Traveler
Nov 2024
Ian
check out.
considering i booked several months in advance i was only informed about refurbishment work between june and 31st december a couple of days before arrival. noise from builders until 8 pm, breakfast not available before 07:30, wasn't informed before hand, never got to eat a breakfast i had booked.
0 star review
Family with Younger Children
Oct 2024
Julie
the location was great, but not for the price.
before we arrived, the hotel asked to change our room from 2 beds to 1 bed because, after i booked, they booked several sports teams that needed rooms with 2 beds. i explained that we needed 2 beds because we were traveling with children. when we arrived, they put us in a room directly under loud construction noise and refused to move us to a different floor. we are hilton honors members and they also refused to upgrade us or discount in any way to address the inhabitable room. the only thing they offered to make us more comfortable was free water.
3 stars review
Family with Younger Children
Oct 2024
Ruth
great location right next to our daughter's university so we could walk to her dorm and move her in. right next to all of the stores and restaurants.
a little expensive.
4 stars review
Solo Traveler
Sep 2024
Colleen
great location, clean and comfortable.
3 stars review
Business Traveler
Sep 2024
Aline
the room is very comfortable
the food service one day didn’t bring one thing that i ordered, and i had no way to complain. the hotel was under construction, and both mornings they started constructing a 7am and 7:30 am.
3 stars review
Family with Younger Children
Sep 2024
Vicky
the lobby is under renovation, making entry and exit inconvenient.
0 star review
Family with Younger Children
Sep 2024
Christopher
staff was friendly and helpful.
upon arrival to the room it looked like it had been chewed up by a pack of dogs. seeing as we had a dog of our own that had opted for a pet stay we decided to overlook that, however immediately we noticed the air conditioner was not cooling the room. we turned it down to 62 and waited a while.. meanwhile i noticed the giant sheets of particle board under the mattresses instead of box springs. also worth noting that though they had noted they were going under lobby renovations we kept having fire drills. ultimately the room would not cool down and we asked to be moved to a different room and we're told that there was no rooms available and they would send up a technician. technician came around 10pm and confirmed the ac was blowing at 61 and that the room would cool down within an hour. at 7:00 a.m. the next morning the room had not cooled down 1°, still at 72. we couldn't sleep at all as we both run hot and prefer temperatures in the 66 to 68 range for sleeping. we only stayed one night of our two nights and i opted for somewhere that wasn't a complete and total waste of time and money.
4 stars review
Solo Traveler
Aug 2024
Brandon
the room
paying another deposit after already prepaying the room!
3 stars review
Business Traveler
Aug 2024
Charles
location
contraction during my stay (loud noise) wifi failure. i had to go to starbucks to work while i would have worked at the property.
1 star review
Business Traveler
Jun 2024
Pan
not really
not desk lamp, call the front desk to request add didn’t get it after 2 days. food there is also not good
1 star review
Family with Younger Children
Jun 2024
Jennifer
the room wasn't the cleanest and the chair in the room had rips all over it.
4 stars review
Solo Traveler
Jun 2024
Melinda
easy, responsive staff. terribly kind and accommodating and professional.
nope; everything was perfect.
3 stars review
Family with Younger Children
May 2024
Smith
location was perfect, downtown evanston. there was a local restaurant near by
there were only 2 outlets in the room
3 stars review
Family with Younger Children
May 2024
Kathy
the breakfast was very good. the staff was friendly and accommodating. beds were comfortable.
this hotel is apparently remodeling several floors. we were assigned to an older floor and arrived to a hot/stuffy room. upon returning from sightseeing and dinner, the a/c still didn't work though maintenance had reported it as fixed. the front desk was able to provide us a fan that made sleeping that night more bearable. after 2nd day full of activities in chicago, arrived back to hotel around midnight to discover a/c still not working. after more late night maintenance visits, staff determined a/c could not be fixed that night. thankfully around 1 am we were moved to an upgraded room at no additional charge with working a/c!! the night manager also removed room charges for that 1st night's stay which we appreciated. the front desk, maintenance, breakfast staffs, etc. were all very nice and courteous. the morning of our last vacation day, we again headed to chicago before housekeeping serviced our room. carelessly, i left my 2nd phone laying on the desk. when we returned to the room that evening, the room had been serviced, but my phone was no longer on the desk. unfortunately i didn't realize the phone was really missing until we checked out the next morning, returned home and was unable to find it in any of our bags, etc. i called the front desk and was told housekeeping did not report any found items left behind. looking back, none of us saw the phone sitting on the desk after returning from chicago (that third sunday night of our stay). i told the front desk i felt confident that it disappeared during our stay and that i would be leaving a review to reflect that. i guess i should have tried leaving it in the safe....but the safe in our 1st room didn't work anyway so i guess we didn't think about trying to use the safe in our 2nd room. i am so disappointed as i had been very happy with how friendly staff had been. i really miss my phone and would like to know where it went.
3 stars review
Business Traveler
May 2024
Anonymous
first 2 nights ac unit of the building not the room, noise drive me crazy at night. changed to the other side for extra 2 night better but the sound proof is not good enough neither.
1 star review
Business Traveler
May 2024
Anonymous
it was close to the college
you had to pay for valet parking and for being a nice hotel the carpet was coming apart when you go in to the bathroom.
5 stars review
Business Traveler
May 2024
Anonymous
beautiful hotel in lovely area.
would love to see free coffee in libby starting at 6am for those on the go - not a fan of the taste of the in-room coffee maker etc
4 stars review
Apr 2024
Mike
staff were extremely friendly and accommodating. property is in walking distance of campus and restaurants.
room was small, especially the bathroom.
4 stars review
Apr 2024
Verified Hotel Guest
the room is clean and comfortable.
the staff at the registration counter is not that friendly.
SEE MAP
Map
COMPARE HOTELS NEARBY
Holiday Inn Chicago North-Evanston, an IHG Hotel
  0.3 
3.75 stars
 Based on 784 reviews
Hilton Garden Inn Chicago North Shore/Evanston
  0.3 
4.05 stars
 Based on 481 reviews
Hyatt House Chicago/Evanston
  0.2 
4.25 stars
 Based on 349 reviews
Graduate by Hilton Evanston
  0.3 
4.1 stars
 Based on 600 reviews
The Gem Museum Suites
  0.4 
 

"""

### Updates

In [7]:
def call_orchestrator(messages, model):
    "Takes in user query -> returns relevant tool to call from the list of available tools."
    
    template = """You are Hotel booking platform orchestrator who can use tools to retrieve relevant information.
        Your objective it to answer user query in the most optimal way and so you must use the tool available to
        to do so. 
        
        Tools available: [construct_multiple_queries, search_web, locality_information, emergency_assessment, generate_answer, done, info_identifier, generate_answer_from_hotel_info]
        construct_multiple_queries: take the input user query and generate multiple optimal queries to search the web.
        search_web: to search the web with the user query for relevant hotels.
        locality_information: provides list of hotels that are closer to the users current location with additional information.
        emergency_assessment: use to answer critical health/ medical related queries but requires output of locality_information to be passed in to assess well.
        generate_answer: to answer user query based on web search results.
        done: to break the workflow when the user query has been successfully answered.
        info_identifier: in case there is ambiguity in terms of nouns, verbs, adjectives and other parts of speech, use to extrapolate the answer and get an updated query with more details
        generate_answer_from_hotel_info: to answer a query based on the information given about a specific hotel
        
        Return type: str
        return only the name of the tool to be used.
        
        History: {messages}"""
    
    print(messages)
    prompt_perspectives = ChatPromptTemplate.from_template(template)
    chain = prompt_perspectives | model
    with get_openai_callback() as cb:
        response = chain.invoke(
            {
                "messages": str(messages),
            }
        )
        print(f"Total Cost (USD): ${format(cb.total_cost, '.6f')}")
    messages.append("Orchestrator output: " + response.content)
    return response.content, cb.total_cost

def check_query(messages, model):
    """Takes in user query and checks if it has all relevant information to book hotel.
        yes -> return to the orchestrator. 
        no -> return to the user."""
    
    print(messages)
    template = """You are a query checker for hotel room booking agent. Your job is to verify if the given user query consist of all relevant information
        to proceed with suggesting a hotel for the user. The query must contain information such as number of days of stay, number of rooms, number of guests etc.
        
        Query: {messages}
        
        Return type: str
        return only True or False where True means the query is sufficient and False represents it is not."""
    
    prompt_perspectives = ChatPromptTemplate.from_template(template)
    chain = prompt_perspectives | model
    with get_openai_callback() as cb:
        response = chain.invoke(
            {
                "messages": str(messages[-2]),
            }
        )
        print(f"Total Cost (USD): ${format(cb.total_cost, '.6f')}")
    messages.append("Query check: " + response.content)
    return response.content, cb.total_cost


def info_identifier(messages, model):
    "Takes in the roles and extracts relevant information from the user query"
    template = """You are an advanced AI assistant specializing in linguistic analysis, text refinement, and **detailed information extrapolation**.  
    Your task is to process the user's query by **expanding on key details, providing additional relevant context**, and returning **only the enhanced and extrapolated version of the query** with improved clarity and structure.

    ### **Process:**
    1. **Named Entity Recognization (NER) & Context Expansion**:  
    - Identify and preserve proper nouns (e.g., names of people, places, organizations).  
    - Recognize temporal expressions (dates, times, durations) and expand their relevance if applicable.  
    - Retain numerical values (quantities, percentages, monetary values) and clarify their significance.  
    - Maintain specialized terms (domain-specific words requiring emphasis) while elaborating on their implications.  

    2. **Part-of-Speech (POS) Analysis & Refinement**:  
    - Ensure precise usage of nouns, pronouns, verbs, adjectives, and adverbs.  
    - Maintain sentence coherence by properly structuring prepositions, conjunctions, and articles.  

    3. **Core Information Extraction & Expansion**:  
    - Identify key ideas and extrapolate additional relevant information to provide a more detailed response.  
    - Remove redundancy while **enhancing completeness** of the original intent.  
    - Ensure logical structure and coherence while expanding descriptions and background context.  

    4. **Rephrasing, Enhancement, and Extrapolation**:  
    - Improve clarity and conciseness while **adding missing details** that would enrich understanding.  
    - Expand abbreviations, provide implicit details explicitly, and ensure a thorough description.  
    - Use a **comprehensive and well-structured narrative** that conveys the most complete possible form of the query.  

    ### **Output Requirement:**  
    - **Return only the extrapolated and enhanced query** as a well-structured, refined, and detailed text.  
    - Do not include explanations, metadata, or intermediate breakdowns—only the final, fully developed query containing all the information you have learned so far.  

    Conversation history: {messages}"""


    # print(messages)
    prompt_perspectives = ChatPromptTemplate.from_template(template)
    chain = prompt_perspectives | model
    with get_openai_callback() as cb:
        response = chain.invoke(
            {
                "messages": str(messages[0]),
            }
        )
        print(f"Total Cost (USD): ${format(cb.total_cost, '.6f')}")
    messages.append("Orchestrator output: " + response.content)
    return response.content, cb.total_cost

def generate_answer_from_hotel_info(messages, query, hotel_info):
    "Takes in Information, Updates user query and generates an answer"

    # print(messages)
    info = hotel_info

    template = """You are an AI assistant responsible for generating answers based on the given information.  
    You must rely strictly on the provided details and the updated user query—**do not use any external knowledge. Answer Exactly what is being asked in {user_query}**  

    ### **Input Sources:**  
    - **Provided Information:** {retrieved_info}  
    - **Updated User Query:** {updated_query}  

    ### **Task:**  
    - Analyze the retrieved information in relation to the updated query.  
    - Generate a **clear, concise, and accurate response** solely based on the given details.  
    - Ensure the response is well-structured and directly addresses the query.  

    ### **Output Requirement:**  
    - **Return only the final generated answer** without any additional commentary or explanation.  

    """

    
    prompt_perspectives = ChatPromptTemplate.from_template(template)
    chain = prompt_perspectives | model
    with get_openai_callback() as cb:
        response = chain.invoke(
            {
                "retrieved_info": str(info),
                "updated_query": str(messages[2]),
                "user_query": messages[0]
            }
        )
        print(f"Total Cost (USD): ${format(cb.total_cost, '.6f')}")
    messages.append("Answer: " + response.content)
    return response.content, cb.total_cost


def get_updated_query(messages):
    "Ask the user to share more information about their booking choice."

    print(messages)
    updated_query = input("The entered information is not sufficient to search for hotels, please share more information like location, dates, number of guests, budget etc.")
    messages.append(updated_query)
    return updated_query


def construct_multiple_queries(messages, model, n = 3):
    """Takes in user query and generate n sub queries which are different versions of the initial user query."""
    
    print("**inside construct multiple queries", messages)
    template = """You are a multi query generator that gets user query as input and generates {n} new queries that can be optimal for web search.
        You must retain all vital information from the initial query in each new query you generate. Do not talk or explain just generate the queries.
        
        Original Query: {messages}
        
        Return type: str
        return {n} queries that fully consists all information about the original query and is optimized for web search. Just return the query, do not number, bullet etc."""
    
    prompt_perspectives = ChatPromptTemplate.from_template(template)
    chain = prompt_perspectives | model 
    with get_openai_callback() as cb:
        response = chain.invoke(
            {
                "messages": str(messages[0]),
                'n': str(n)
            }
        )
        print(f"Total Cost (USD): ${format(cb.total_cost, '.6f')}")
    messages.append("Query check: " + response.content)
    return response.content, cb.total_cost


def search_ddg(sub):
    "Takes in user query -> searches the web -> return relevant information."
    search_query = sub

    def fetch_full_text(url):
        try:
            response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=10)
            response.raise_for_status()  # Raise an error for bad responses
            soup = BeautifulSoup(response.text, "html.parser")
            
            # Extract main text content from <p> tags
            paragraphs = soup.find_all("p")
            full_text = "\n".join([p.get_text() for p in paragraphs])
            return full_text
        except requests.exceptions.RequestException as e:
            return f"Error fetching page: {e}"

    # Function to search DuckDuckGo
    def search_duckduckgo(query, num_results=3):
        searched_output = []
        with DDGS() as ddgs:
            results = list(ddgs.text(query, max_results=num_results))
        for i, search_results in enumerate(results, 1):
            link_text = fetch_full_text(search_results['href'])
            result = f"\nSearch: {i}\nTitle: {search_results['title']}\nBody: {link_text}"
            searched_output.append(result)
        return searched_output
    
    search_results = search_duckduckgo(search_query, num_results=5)

    return search_results

def emergency_assessment(messages, additional_information):
    """Takes in user query, understand user needs and answer based on their emergency. Requires other tools to provide information based on locality.
        Returns an answer that would best assess the user condition."""
    
    print(messages)
    template = """Your primary function is to help users find the best hotel.
        In general, consider user preferences like sanitation and reviews. 
        HOWEVER, in emergency situations, the user's immediate need is paramount.  
        Therefore, if the context is an emergency, prioritize hotels based on their distance to the user above all other factors. Explain to the user why distance is being prioritized in this emergency.   
        
        Query: {messages}
        
        Additional Information: {additional_information}

        Return type: str
        Return an answer that would best assess the user condition."""
    
    prompt_perspectives = ChatPromptTemplate.from_template(template)
    chain = prompt_perspectives | model
    with get_openai_callback() as cb:
        response = chain.invoke(
            {
                "additional_information": str(additional_information), 
                "messages": str(messages[-4])
            }
        )
        print(f"Total Cost (USD): ${format(cb.total_cost, '.6f')}")
    messages.append("Emergency assessment: " + response.content)
    return response.content, cb.total_cost

def locality_information(messages):
    """Gives list of hotels that are closer to the user with additional information."""
    
    hotel_list = {
        "Hotel 1": {
            "Name": "Marriott Hotel",
            "Review": "2.0",
            "Popular positive comment": "Hotel is affordable",
            "Popular negative comment": "Nil",
            "GPS Distance from Clara": "0.5 miles",
        },
        "Hotel 2" : {
            "Name": "AVASA Hotel",
            "Review": "1.5",
            "Popular positive comment": "Food is good",
            "Popular negative comment": "Rooms are old",
            "GPS Distance from Clara": "0.2 miles",
        },
        "Hotel 3": {
            "Name": "Lemontree Hotel",
            "Review":"3.5",
            "Popular positive comment": "Good for a family with kids",
            "Popular negative comment": "Swimming pool was not clean",
            "GPS Distance from Clara": "1.5 miles",
        },
        "Hotel 4": {
            "Name": "Westin Hotel",
            "Review":"4.0",
            "Popular positive comment": "Room service was good",
            "Popular negative comment": "Nil",
            "GPS Distance from Clara": "1 mile",
        }
    }
    messages.append("Locality information:" + str(hotel_list))
    return hotel_list

def generate_answer(messages, query):
    "Based on the obtained search results answer the user query"

    print(messages)
    web_search = messages[-2]
    # Even if the user query restricts you to only give binary answers, if there exist an hotel that satisfies some of the user needs then reply with it.
    # Remember the objective is to satisfy the customer and so an answer with just binary answer won't in achieve that goal.
    template = """You are a hotel booking platform QA bot employed to answer user query based on retrieved information. Do not use your own knowledge
        but rely only on the extracted information. If the extracted information doesn't answer all question, it's okay, provide a concise version of the hotels that
        satisfies at least some of the the users request.
        
        Web search: {web_search}
        
        Query: {query}"""

    prompt_perspectives = ChatPromptTemplate.from_template(template)
    # print(prompt_perspectives)
    chain = prompt_perspectives | model
    with get_openai_callback() as cb:
        response = chain.invoke(
            {
                "web_search": str(web_search),
                "query": str(query)
            }
        )
        print(f"Total Cost (USD): ${format(cb.total_cost, '.6f')}")
    messages.append("Answer: " + response.content)
    return response.content, cb.total_cost

def done(messages):
    "Used as an indicator to terminate the program."
    
    if "done" in messages[-1]:
        return True


In [8]:
import requests
import json
from datetime import datetime, timedelta
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.callbacks import get_openai_callback

def search(user_query, model):
    """
    Takes in a user query, extracts location and booking details, and returns hotel information.
    
    Args:
        user_query (str): The user's hotel search query
        openai_api_key (str): OpenAI API key for LLM processing
        
    Returns:
        dict: Hotel search results from booking.com API
    """
    
    # Extract location information using LLM
    location_info = extract_location(user_query, model)
    
    # Get destination ID using the first API
    dest_id = get_destination_id(location_info)
    
    # Extract booking parameters using LLM
    booking_params = extract_booking_params(user_query, dest_id, model)
    
    # Get hotel information using the second API
    hotels = get_hotel_information(booking_params)
    
    return hotels

def extract_location(user_query, model):
    """Extract location information from user query using LLM"""
    template = """
    Extract the destination/location information from the following hotel booking query.
    If no location is mentioned, return "chicago" as the default.
    
    Query: {query}
    
    Return only the location name, nothing else.
    """
    
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | model
    
    with get_openai_callback() as cb:
        location = chain.invoke({"query": user_query}).content.strip().lower()
        print(f"Location extraction cost: ${format(cb.total_cost, '.6f')}")
    
    return location

def get_destination_id(location):
    """Get destination ID from booking.com API"""
    url = "https://booking-com15.p.rapidapi.com/api/v1/hotels/searchDestination"
    
    querystring = {"query": location}
    headers = {
        "x-rapidapi-key": "1a7b208851msha50d62011abf87bp1ccb15jsnfb7159b24ba2",
        "x-rapidapi-host": "booking-com15.p.rapidapi.com"
    }
    
    try:
        response = requests.get(url, headers=headers, params=querystring)
        response.raise_for_status()
        data = response.json()
        
        # Get the first destination from the results
        if data and "data" in data and len(data["data"]) > 0:
            dest_id = data["data"][0]["dest_id"]
            print(f"Found destination ID: {dest_id} for location: {location}")
            return dest_id
        else:
            # Default to Chicago destination ID if location not found
            print(f"Location '{location}' not found, defaulting to Chicago (20033173)")
            return "20033173"  # Chicago's dest_id
    
    except Exception as e:
        print(f"Error getting destination ID: {str(e)}")
        return "20033173"  # Default to Chicago's dest_id

def extract_booking_params(user_query, dest_id, model):
    """Extract booking parameters from user query using LLM"""
    # Get current date and next day for default values
    today = datetime.now()
    tomorrow = today + timedelta(days=1)
    default_arrival = today.strftime("%Y-%m-%d")
    default_departure = tomorrow.strftime("%Y-%m-%d")
    
    template = """
    Extract hotel booking parameters from the following query.
    
    Query: {query}
    
    Return a JSON object with the following fields:
    - arrival_date: in YYYY-MM-DD format (if not specified, use {default_arrival})
    - departure_date: in YYYY-MM-DD format (if not specified, use {default_departure})
    - adults: number of adults (default: 1)
    - children_age: comma-separated list of ages (default: "0,17" if children mentioned but ages not specified)
    - room_qty: number of rooms (default: 1)
    - price_min: minimum price in USD (optional)
    - price_max: maximum price in USD (optional)
    
    Return ONLY the JSON object, nothing else.
    """
    
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | model
    
    with get_openai_callback() as cb:
        result = chain.invoke({
            "query": user_query,
            "default_arrival": default_arrival,
            "default_departure": default_departure
        }).content.strip()
        print(f"Parameter extraction cost: ${format(cb.total_cost, '.6f')}")
    
    try:
        result = result.strip("```").strip("json")
        print(result)
        params = json.loads(result)
    except json.JSONDecodeError:
        print("Error parsing LLM response as JSON, using default parameters")
        params = {
            "arrival_date": default_arrival,
            "departure_date": default_departure,
            "adults": 1,
            "children_age": "0,17",
            "room_qty": 1
        }
    
    # Build complete parameter set with the extracted dest_id
    booking_params = {
        "dest_id": dest_id,
        "search_type": "CITY",
        "arrival_date": params.get("arrival_date", default_arrival),
        "departure_date": params.get("departure_date", default_departure),
        "adults": str(params.get("adults", 1)),
        "room_qty": str(params.get("room_qty", 1)),
        "units": "metric",
        "temperature_unit": "c",
        "languagecode": "en-us"
    }
    
    # Add optional parameters if present
    if "children_age" in params and params["children_age"]:
        booking_params["children_age"] = params["children_age"]
    
    if "price_min" in params and params["price_min"]:
        booking_params["price_min"] = str(params["price_min"])
    
    if "price_max" in params and params["price_max"]:
        booking_params["price_max"] = str(params["price_max"])
    
    return booking_params

def get_hotel_information(params):
    """Get hotel information from booking.com API"""
    url = "https://booking-com15.p.rapidapi.com/api/v1/hotels/searchHotels"
    
    headers = {
        "x-rapidapi-key": "1a7b208851msha50d62011abf87bp1ccb15jsnfb7159b24ba2",
        "x-rapidapi-host": "booking-com15.p.rapidapi.com"
    }
    
    try:
        print(f"Searching hotels with parameters: {params}")
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        return response.json()
    
    except Exception as e:
        print(f"Error getting hotel information: {str(e)}")
        return {"error": str(e)}


In [9]:
### Question to ask:
## Q1:
# suggest me hotels for a stay of three days for 4 guests between the budget of 900$ and 1200$. The hotel have a more than 30 floors, at least 1 swimming pool, 2 rooms must include air conditioning and heating with a dressing room and bathroom.  make sure the room overlooks a beach. if you're able to find such hotel then reply with a yes, otherwise no and nothing else.
# suggest me hotels for a stay of three days for 4 guests between the budget of 900$ and 1200$. The hotel has at least 1 swimming pool and 2 rooms. if you're able to find such hotel then reply with a yes, otherwise no and nothing else.

## Q2:
# Consider you are the manager of Hotel Orrington. On a fine day the bus named infinity bus consisting of people named “infinity times 1” to “infinity times 10”. Let us know if you can accommodate the people travelling in the infinity bus. "

## Q3:
# Clara is extremely meticulous when it comes to cleanliness and expects everything she uses and everywhere she stays to be well organised and tidy. Clara is in an emergency situation and needs to use the room for medical purposes. Your task is to suggest a hotel for clara.

In [10]:
completed_task = False
messages = []
query = input("Hi, how can I assist you.") # hotels with 3 rooms 2 bathroom for 3 people for 3 nights
messages.append(query)

model = AzureChatOpenAI(azure_deployment="gpt-4o", api_version="2024-10-21",  temperature=0, max_tokens=512, timeout=None, max_retries=2)

while not completed_task:
    choice_tool, _ = call_orchestrator(messages, model)    
    
    if choice_tool == "construct_multiple_queries":
        response, _ = construct_multiple_queries(messages, model)
        messages.append(response)
    
    elif choice_tool == "search_web":
        temp = []
        sub_query = messages[-2].split("\n\n")
        for sub in sub_query:
            web_results = search(sub, model)
            for i in web_results:
                temp.append(i)
        messages.append(temp)
        print("*****FINISHED RUNNING SEARCH")

    elif choice_tool == "generate_answer":
        response = generate_answer(messages, query)
    
    # extract locality information
    elif choice_tool == "locality_information":
        print('Hello')
        locality_info = locality_information(messages)
    
    elif choice_tool == "emergency_assessment":
        response, _ = emergency_assessment(messages, locality_info)
    
    elif choice_tool == "info_identifier":
        response, _ = info_identifier(messages, model)
   

    elif choice_tool == "generate_answer_from_hotel_info":
        response, _ = generate_answer_from_hotel_info(messages, query, HOTEL_INFO)
        completed_task = True
        
    elif choice_tool == "done":
        print("Inside ELIF")
        completed_task = done(messages)
        # print(completed_task)

["suggest me hotels for a stay of three days for 4 guests between the budget of 900$ and 1200$. The hotel has at least 1 swimming pool and 2 rooms. if you're able to find such hotel then reply with a yes, otherwise no and nothing else."]
Total Cost (USD): $0.000885
**inside construct multiple queries ["suggest me hotels for a stay of three days for 4 guests between the budget of 900$ and 1200$. The hotel has at least 1 swimming pool and 2 rooms. if you're able to find such hotel then reply with a yes, otherwise no and nothing else.", 'Orchestrator output: construct_multiple_queries']
Total Cost (USD): $0.001260
["suggest me hotels for a stay of three days for 4 guests between the budget of 900$ and 1200$. The hotel has at least 1 swimming pool and 2 rooms. if you're able to find such hotel then reply with a yes, otherwise no and nothing else.", 'Orchestrator output: construct_multiple_queries', 'Query check: Hotels with 2 rooms and a swimming pool for 4 guests, 3-day stay, budget betwe

In [26]:
for i in messages:
    print(i)


suggest me hotels for a stay of three days for 4 guests between the budget of 900$ and 1200$. The hotel has at least 1 swimming pool and 2 rooms. if you're able to find such hotel then reply with a yes, otherwise no and nothing else.
Orchestrator output: construct_multiple_queries
Query check: Hotels with 2 rooms and a swimming pool for 4 guests, 3-day stay, budget between $900-$1200.  

Find hotels with at least 1 swimming pool, 2 rooms, accommodating 4 guests for 3 days, budget $900-$1200.  

3-day stay hotels for 4 guests, budget $900-$1200, with 2 rooms and a swimming pool.  
Hotels with 2 rooms and a swimming pool for 4 guests, 3-day stay, budget between $900-$1200.  

Find hotels with at least 1 swimming pool, 2 rooms, accommodating 4 guests for 3 days, budget $900-$1200.  

3-day stay hotels for 4 guests, budget $900-$1200, with 2 rooms and a swimming pool.  
Orchestrator output: search_web
['status', 'message', 'timestamp', 'data', 'status', 'message', 'timestamp', 'data', 'sta

In [25]:
messages[-1]

'Orchestrator output: done'

### Trying hotel api

In [16]:
import requests

url = "https://booking-com15.p.rapidapi.com/api/v1/hotels/searchDestination"

querystring = {"query":"chicago"}

headers = {
	"x-rapidapi-key": "1a7b208851msha50d62011abf87bp1ccb15jsnfb7159b24ba2",
	"x-rapidapi-host": "booking-com15.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'status': True, 'message': 'Success', 'timestamp': 1741666920252, 'data': [{'dest_id': '20033173', 'search_type': 'city', 'roundtrip': 'GhAxMTU0MWViNDFmNjgwNDEwIAAoATICZW46B2NoaWNhZ29AAEoAUAA=', 'hotels': 797, 'type': 'ci', 'label': 'Chicago, Illinois, United States', 'image_url': 'https://cf.bstatic.com/xdata/images/city/150x150/689744.jpg?k=3c7c6d59d968c2bade9003d9ebf8a1346455ce92abe94ffaa29247839161f09e&o=', 'cc1': 'us', 'dest_type': 'city', 'city_ufi': None, 'region': 'Illinois', 'name': 'Chicago', 'country': 'United States', 'lc': 'en', 'nr_hotels': 797, 'city_name': 'Chicago', 'longitude': -87.6243, 'latitude': 41.8821}, {'dest_id': '2614', 'search_type': 'district', 'roundtrip': 'GhAxMTU0MWViNDFmNjgwNDEwIAEoATICZW46B2NoaWNhZ29AAEoAUAA=', 'hotels': 261, 'label': 'Downtown Chicago, Chicago, Illinois, United States', 'type': 'di', 'image_url': 'https://cf.bstatic.com/xdata/images/district/150x150/55543.jpg?k=c98fd30142aad073de2cf65c1bd05877e0519f5dc195e804e593dee441fb0641&o=', 'cc

In [20]:
import requests

url = "https://booking-com15.p.rapidapi.com/api/v1/hotels/searchHotels"

querystring = {"dest_id":"20033173","search_type":"CITY","arrival_date":"2025-03-10", "departure_date":"2025-03-11","price_min":"900","adults":"1","children_age":"0,17","room_qty":"1","units":"metric","temperature_unit":"c","languagecode":"en-us"}

headers = {
	"x-rapidapi-key": "1a7b208851msha50d62011abf87bp1ccb15jsnfb7159b24ba2",
	"x-rapidapi-host": "booking-com15.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'status': True, 'message': 'Success', 'timestamp': 1741667059995, 'data': {'hotels': [{'hotel_id': 2342582, 'accessibilityLabel': 'Viceroy Chicago.\n5 out of 5 stars.\n9.0 Wonderful 1364 reviews.\n\u200eNear North Side\u202c • \u200e2.3 km from downtown\u202c.\n Private suite – 63\xa0m² : 2\xa0beds • 1\xa0bedroom • 1\xa0living room • 1\xa0bathroom.\n995 USD.\n+200 USD taxes and charges.\nNo prepayment needed.', 'property': {'photoUrls': ['https://cf.bstatic.com/xdata/images/hotel/square500/522821988.jpg?k=c58ccdbfa8e531d6e3cfadbdba1286c67e0333f91d046d7510ec38af52de9f94&o=', 'https://cf.bstatic.com/xdata/images/hotel/square1024/522821988.jpg?k=c58ccdbfa8e531d6e3cfadbdba1286c67e0333f91d046d7510ec38af52de9f94&o=', 'https://cf.bstatic.com/xdata/images/hotel/square2000/522821988.jpg?k=c58ccdbfa8e531d6e3cfadbdba1286c67e0333f91d046d7510ec38af52de9f94&o='], 'blockIds': ['234258210_397937265_2_0_0'], 'id': 2342582, 'checkoutDate': '2025-03-11', 'latitude': 41.9024600833473, 'checkout': {'until

In [ ]:
response.json()['data']['hotels'][:10]

In [14]:
import requests
import json
from datetime import datetime, timedelta
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.callbacks import get_openai_callback

def search_hotels_from_query(user_query, model):
    """
    Takes in a user query, extracts location and booking details, and returns hotel information.
    
    Args:
        user_query (str): The user's hotel search query
        openai_api_key (str): OpenAI API key for LLM processing
        
    Returns:
        dict: Hotel search results from booking.com API
    """
    
    # Extract location information using LLM
    location_info = extract_location(user_query, model)
    
    # Get destination ID using the first API
    dest_id = get_destination_id(location_info)
    
    # Extract booking parameters using LLM
    booking_params = extract_booking_params(user_query, dest_id, model)
    
    # Get hotel information using the second API
    hotels = get_hotel_information(booking_params)
    
    return hotels

def extract_location(user_query, model):
    """Extract location information from user query using LLM"""
    template = """
    Extract the destination/location information from the following hotel booking query.
    If no location is mentioned, return "chicago" as the default.
    
    Query: {query}
    
    Return only the location name, nothing else.
    """
    
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | model
    
    with get_openai_callback() as cb:
        location = chain.invoke({"query": user_query}).content.strip().lower()
        print(f"Location extraction cost: ${format(cb.total_cost, '.6f')}")
    
    return location

def get_destination_id(location):
    """Get destination ID from booking.com API"""
    url = "https://booking-com15.p.rapidapi.com/api/v1/hotels/searchDestination"
    
    querystring = {"query": location}
    headers = {
        "x-rapidapi-key": "1a7b208851msha50d62011abf87bp1ccb15jsnfb7159b24ba2",
        "x-rapidapi-host": "booking-com15.p.rapidapi.com"
    }
    
    try:
        response = requests.get(url, headers=headers, params=querystring)
        response.raise_for_status()
        data = response.json()
        
        # Get the first destination from the results
        if data and "data" in data and len(data["data"]) > 0:
            dest_id = data["data"][0]["dest_id"]
            print(f"Found destination ID: {dest_id} for location: {location}")
            return dest_id
        else:
            # Default to Chicago destination ID if location not found
            print(f"Location '{location}' not found, defaulting to Chicago (20033173)")
            return "20033173"  # Chicago's dest_id
    
    except Exception as e:
        print(f"Error getting destination ID: {str(e)}")
        return "20033173"  # Default to Chicago's dest_id

def extract_booking_params(user_query, dest_id, model):
    """Extract booking parameters from user query using LLM"""
    # Get current date and next day for default values
    today = datetime.now()
    tomorrow = today + timedelta(days=1)
    default_arrival = today.strftime("%Y-%m-%d")
    default_departure = tomorrow.strftime("%Y-%m-%d")
    
    template = """
    Extract hotel booking parameters from the following query.
    
    Query: {query}
    
    Return a JSON object with the following fields:
    - arrival_date: in YYYY-MM-DD format (if not specified, use {default_arrival})
    - departure_date: in YYYY-MM-DD format (if not specified, use {default_departure})
    - adults: number of adults (default: 1)
    - children_age: comma-separated list of ages (default: "0,17" if children mentioned but ages not specified)
    - room_qty: number of rooms (default: 1)
    - price_min: minimum price in USD (optional)
    - price_max: maximum price in USD (optional)
    
    Return ONLY the JSON object, nothing else.
    """
    
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | model
    
    with get_openai_callback() as cb:
        result = chain.invoke({
            "query": user_query,
            "default_arrival": default_arrival,
            "default_departure": default_departure
        }).content.strip()
        print(f"Parameter extraction cost: ${format(cb.total_cost, '.6f')}")
    
    try:
        result = result.strip("```").strip("json")
        print(result)
        params = json.loads(result)
    except json.JSONDecodeError:
        print("Error parsing LLM response as JSON, using default parameters")
        params = {
            "arrival_date": default_arrival,
            "departure_date": default_departure,
            "adults": 1,
            "children_age": "0,17",
            "room_qty": 1
        }
    
    # Build complete parameter set with the extracted dest_id
    booking_params = {
        "dest_id": dest_id,
        "search_type": "CITY",
        "arrival_date": params.get("arrival_date", default_arrival),
        "departure_date": params.get("departure_date", default_departure),
        "adults": str(params.get("adults", 1)),
        "room_qty": str(params.get("room_qty", 1)),
        "units": "metric",
        "temperature_unit": "c",
        "languagecode": "en-us"
    }
    
    # Add optional parameters if present
    if "children_age" in params and params["children_age"]:
        booking_params["children_age"] = params["children_age"]
    
    if "price_min" in params and params["price_min"]:
        booking_params["price_min"] = str(params["price_min"])
    
    if "price_max" in params and params["price_max"]:
        booking_params["price_max"] = str(params["price_max"])
    
    return booking_params

def get_hotel_information(params):
    """Get hotel information from booking.com API"""
    url = "https://booking-com15.p.rapidapi.com/api/v1/hotels/searchHotels"
    
    headers = {
        "x-rapidapi-key": "1a7b208851msha50d62011abf87bp1ccb15jsnfb7159b24ba2",
        "x-rapidapi-host": "booking-com15.p.rapidapi.com"
    }
    
    try:
        print(f"Searching hotels with parameters: {params}")
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        return response.json()
    
    except Exception as e:
        print(f"Error getting hotel information: {str(e)}")
        return {"error": str(e)}


In [22]:
# Example usage

user_query = "I need a hotel in  for 2 adults. Budget around $300 per night."

model = AzureChatOpenAI(azure_deployment="gpt-4o", api_version="2024-10-21",  temperature=0, max_tokens=512, timeout=None, max_retries=2)
results = search_hotels_from_query(user_query, model)
print(json.dumps(results, indent=2))

Location extraction cost: $0.000197
Found destination ID: 20033173 for location: chicago
Parameter extraction cost: $0.001200

{
    "arrival_date": "2025-03-11",
    "departure_date": "2025-03-12",
    "adults": 2,
    "children_age": "0,17",
    "room_qty": 1,
    "price_min": null,
    "price_max": 300
}

Searching hotels with parameters: {'dest_id': '20033173', 'search_type': 'CITY', 'arrival_date': '2025-03-11', 'departure_date': '2025-03-12', 'adults': '2', 'room_qty': '1', 'units': 'metric', 'temperature_unit': 'c', 'languagecode': 'en-us', 'children_age': '0,17', 'price_max': '300'}
{
  "status": true,
  "message": "Success",
  "timestamp": 1741724245527,
  "data": {
    "hotels": [
      {
        "hotel_id": 329658,
        "accessibilityLabel": "Graduate by Hilton Evanston.\n4 out of 5 stars.\n8.5 Very Good 705 reviews.\n\u200e19 km from downtown\u202c.\n2\u00a0beds.\n209 USD.\n+30 USD taxes and charges.\nNo prepayment needed.",
        "property": {
          "reviewScore":

In [10]:
a = """```json{  "arrival_date": "2025-03-15",  "departure_date": "2025-03-20",  "adults": 2,  "children_age": "0,17",  "room_qty": 1,  "price_min": null,  "price_max": 300}```"""

In [46]:
json.loads(a.strip("```").strip("json"))

{'arrival_date': '2025-03-15',
 'departure_date': '2025-03-20',
 'adults': 2,
 'children_age': '0,17',
 'room_qty': 1,
 'price_min': None,
 'price_max': 300}